# Tutorial de Llamadas Paralelas a Funciones con Phi-4 Mini ONNX

Este cuaderno demuestra cómo usar Phi-4 Mini con ONNX Runtime GenAI para realizar llamadas paralelas a funciones. Las llamadas a funciones permiten que el modelo invoque herramientas y APIs externas de manera inteligente según las solicitudes del usuario.

## Descripción General

En este tutorial, aprenderás a:
- Configurar Phi-4 Mini con ONNX Runtime GenAI
- Definir esquemas de funciones para reservar vuelos y hoteles
- Utilizar generación guiada con gramática Lark para obtener salidas estructuradas
- Ejecutar llamadas paralelas a funciones para escenarios complejos de reserva de viajes

## Requisitos Previos

Antes de ejecutar este cuaderno, asegúrate de:
- Haber descargado el modelo Phi-4 Mini ONNX
- Tener instalado el paquete `onnxruntime-genai`
- Comprender los conceptos básicos de llamadas a funciones


## Paso 1: Importar las bibliotecas necesarias

Primero, importaremos las bibliotecas necesarias para la implementación de nuestra función.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Paso 2: Configuración y Configuración del Modelo

Ahora configuraremos el modelo Phi-4 Mini ONNX. Asegúrate de reemplazar la ruta con el directorio real de tu modelo.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Paso 3: Configurar los parámetros de generación

Configura los parámetros de generación para controlar el comportamiento de salida del modelo. Estos ajustes garantizan respuestas deterministas y enfocadas para la llamada de funciones.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Paso 4: Definir las Funciones Disponibles

Aquí definimos las funciones que nuestro asistente de IA puede utilizar. En este ejemplo, tenemos dos funciones relacionadas con viajes:
1. **booking_flight_tickets**: Para reservar vuelos entre aeropuertos
2. **booking_hotels**: Para reservar alojamientos en hoteles

Las definiciones de las funciones siguen el formato de esquema de llamadas de funciones de OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Paso 5: Funciones auxiliares para la generación de gramática

Estas funciones auxiliares convierten nuestras definiciones de funciones al formato de gramática de Lark, que es utilizado por ONNX Runtime GenAI para generación guiada. Esto garantiza que el modelo produzca llamadas de función válidas en el formato JSON correcto.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Paso 6: Probar la generación de gramática

Probemos nuestras funciones de generación de gramática para ver cómo convierten las definiciones de nuestras herramientas al formato adecuado.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Paso 7: Preparar el mensaje del sistema y el generador

Ahora crearemos el mensaje del sistema que informa al modelo sobre las herramientas disponibles y configuraremos el generador con parámetros de generación guiada.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Paso 8: Inicializar el Generador con Generación Guiada

Ahora crearemos el generador con nuestros parámetros configurados y aplicaremos la gramática Lark para la generación guiada.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Paso 9: Probar la llamada paralela de funciones

Ahora probemos nuestra configuración con un escenario complejo de reserva de viajes que requiere llamar a múltiples funciones.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Generar Llamadas a Funciones

El modelo ahora generará llamadas a funciones estructuradas basadas en la solicitud del usuario. Gracias a la generación guiada, la salida estará en formato JSON válido que se puede ejecutar directamente.

**Salida Esperada**: El modelo debería generar llamadas a funciones para:
1. `booking_flight_tickets` con detalles de Beijing (PEK) a París (CDG)
2. `booking_hotels` con detalles de alojamiento en París

Ejecuta la celda a continuación para ver la generación en vivo:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Conclusión

🎉 **¡Felicidades!** Has implementado con éxito llamadas paralelas a funciones con Phi-4 Mini utilizando ONNX Runtime GenAI.

### Lo que has aprendido:

1. **Configuración del modelo**: Cómo configurar Phi-4 Mini con ONNX Runtime GenAI
2. **Definición de funciones**: Cómo definir esquemas de funciones para llamadas a funciones de IA
3. **Generación guiada**: Cómo usar la gramática de Lark para generar salidas estructuradas
4. **Llamadas paralelas a funciones**: Cómo manejar solicitudes complejas que requieren múltiples llamadas a funciones

### Beneficios clave:

- ✅ **Salida estructurada**: La generación guiada asegura llamadas a funciones JSON válidas
- ✅ **Procesamiento paralelo**: Maneja múltiples llamadas a funciones en una sola solicitud
- ✅ **Alto rendimiento**: ONNX Runtime proporciona inferencia optimizada
- ✅ **Esquema flexible**: Fácil de agregar o modificar definiciones de funciones

### Recursos:

- [Documentación de Phi-4 Mini](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [Documentación de ONNX Runtime GenAI](https://onnxruntime.ai/docs/genai/)
- [Mejores prácticas para llamadas a funciones](https://platform.openai.com/docs/guides/function-calling)



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
